# Seperating stock to forcast for certain dates depending on the product

In [1]:
%pip install nbformat


[notice] A new release of pip is available: 24.1.1 -> 25.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [3]:
# import datasets and transformations (reusability purposes)
from IPython import get_ipython

get_ipython().run_line_magic('run', 'datasets.ipynb')

/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_68261/1341293601.py:4: DtypeWarning: Columns (4,6,20,50,52) have mixed types. Specify dtype option on import or set low_memory=False.
  products_s = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] Products [STOCK].txt', sep='\t', header=0) # stock
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_68261/1341293601.py:5: DtypeWarning: Columns (5,48) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_i = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] Tickets [CUSTOM] [ITEMS].txt', sep='\t', header=0) # customer order items
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_68261/1341293601.py:6: DtypeWarning: Columns (43,57,105,140,211,246,253,291,295,312,323,326,328,346,350) have mixed types. Specify dtype option on import or set low_memory=False.
  tickets_c_m = pd.read_csv('./datasets/stock_forecasting/2022-2025/[LT] Tickets [CUSTOM] [MAIN].txt', sep='\t', h

-------------- Product [STOCK] ----------------
   Adhesive  Alternate  Available  BackOrdered Box_Size  CaseQty Color  \
0       NaN        NaN          0            0      NaN      NaN   NaN   
1       NaN        NaN          0            0      NaN      NaN   NaN   
2       NaN        NaN          0            0      NaN      NaN   NaN   
3       NaN        NaN          0            0      NaN      NaN   NaN   
4       NaN        NaN          0            0      NaN      NaN   NaN   

   Commission   Cost  Currency_ExchangeRate  ...  SupplierName  SupplierNo  \
0           0   0.00                      0  ...           NaN         NaN   
1           0   0.00                      0  ...           NaN         NaN   
2           0   0.00                      0  ...           NaN         NaN   
3           0   0.00                      0  ...           NaN         NaN   
4           0  49.11                      0  ...           NaN         NaN   

  SupplierNotes SupplierPartNo Tag Tot

/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_68261/3936327507.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'], errors='coerce')
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_68261/3936327507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_data['OrderDate'] = pd.to_datetime(merged_data['OrderDate'], errors='coerce')
/var/folders/31/9bd8ksys1rzbjk8qs9scl95h0000gn/T/ipykernel_68261/3936327507.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`

In [4]:
# create solution to add each first three unique values of the column 'ProductNumber' to a list
products_by_customer = set()

# loop through all product numbers
for index, row in product_sales.iterrows():
    product = row['ProductNumber']
    for i in range(len(product) - 2):
        customer_code = product[i:i+3]  # get the first three characters of the product number

        # add the customer code to the set
        products_by_customer.add(customer_code)

        # create a list for each customer code / if it exists, add it to the existing customer code list
        if f'custom_code_{customer_code}' in locals():
            locals()[f'custom_code_{customer_code}'].append(row.to_dict())
        else:
            locals()[f'custom_code_{customer_code}'] = [row.to_dict()]

print(f"All custom codes for existing products: \n{products_by_customer} \n ------------------------------------")

# Example to print one of the custom code lists
if 'custom_code_FRE' in locals():
    print(f"Records for custom code FRE: \n{locals()['custom_code_FRE']}")

All custom codes for existing products: 
{'871', '586', '775', '98W', '23S', '149', '583', 'IDA', '570', 'RP-', '11V', '792', '40Z', '10V', '91X', '3X-', 'USU', '89X', '999', '84V', 'ED-', 'DOH', '853', 'PLE', '35W', '952', '724', '301', '82X', '67Y', 'MCC', '177', 'MA-', '824', '08U', '780', '61T', '60U', '19X', '751', '53V', '90X', '085', 'GLO', '036', '17V', '673', '392', '15T', '45W', '063', '450', '279', 'NAF', '03Y', '886', '383', 'HEA', '78X', '3Z-', '7Z-', 'R-F', '71Y', 'CON', '83X', '137', '431', '656', 'AR-', '57W', '206', '633', '79Z', '446', '649', '655', 'MEY', '68T', '263', '801', '942', 'WEI', 'SPW', '332', '407', '976', '498', '193', '336', '056', '47Z', 'N-T', '590', '062', '575', '72W', '894', '489', '97X', '253', '905', '68Y', '96Z', '70Z', 'JMP', '919', '52W', '617', '802', 'RU-', '08S', 'SVO', 'ISE', '30-', '972', '31V', 'JUR', '93X', '33U', '-03', '2-0', '953', '84X', '07-', '026', 'ORC', '126', '029', 'IT-', '24X', '561', '167', 'INF', '764', 'JF-', '820', '510',

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# define the features and target variable from 'product_sales'
X = custom_code_FRE[['ProductNumber', 'order_month']]
y = custom_code_FRE['OrderQuantity']

# convert 'ProductNumber' to numeric values
label_encoder = LabelEncoder()
X['ProductNumber'] = label_encoder.fit_transform(X['ProductNumber'])

# normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# train-test split with scaled features
X_train_scaled, X_val_scaled, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(criterion='squared_error', max_depth=20, min_samples_leaf=2,random_state=42)
rf.fit(X_train_scaled, y_train)
y_pred_rf = rf.predict(X_val_scaled)

In [ ]:
evaluate_model(rf, X_scaled, y)
print('--------------------------------------------')